# Realtime Enemy Detector in Call of Duty

First we need to import the base overlay from Pynq and set up our HDMI I/O.

In [2]:
# based off of https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/notebooks/video/opencv_face_detect_hdmi.ipynb

from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [3]:
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [4]:
hdmi_in.tie(hdmi_out)

Next we can import the libraries for image processing, and define a function for drawing shapes on an image.

In [4]:
import cv2
import numpy as np

def draw_rectangle(draw_point):
    cv2.rectangle(clip, draw_point, (draw_point[0] + 80, draw_point[1] + 80), (0, 0, 255), 0)

Next we can set up OpenCV's "Simple Blob Detector" and configure the parameters it will use.

In [5]:
params = cv2.SimpleBlobDetector_Params()

params.minThreshold = 50
params.maxThreshold = 80

params.filterByArea = True
params.minArea = 120
#params.maxArea = 300

params.filterByCircularity = False
params.minCircularity = 0.1

params.filterByConvexity = True
params.minConvexity = 0.87

params.filterByInertia = False
params.minInertiaRatio = 0.01

detector = cv2.SimpleBlobDetector_create(params)

Next we define our function for processing the frame, making it easier for the blob detector to find blobs.

In [6]:
def process_image(frame):
    orig = frame
#     cv2.rectangle(frame, (50,50), (150,150), (255), thickness = 3)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.blur(gray, ksize=(25,25))
    ret,th = cv2.threshold(blur,30,255,cv2.THRESH_BINARY)
    return blur

In [7]:
gray = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
blur = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, 
                           hdmi_in.mode.width), dtype=np.uint8)

while True:
    clip = hdmi_in.readframe()
    cv2.cvtColor(clip, cv2.COLOR_BGR2GRAY, dst=gray)
    cv2.blur(gray, ksize=(25,25), dst=blur)
    clip.freebuffer()
    outframe = hdmi_out.newframe()
    cv2.threshold(blur,30,255,cv2.THRESH_BINARY, dst=outframe)
    hdmi_out.writeframe(outframe)

KeyboardInterrupt: 

Finally we run the main runloop to show the processed image on HDMI out.

In [7]:
import time

numframes = 480
start = time.time()
timestamp = time.time()
timeReadingFrames = 0
timeBlurring = 0
timeKeypoints = 0
timeDrawingPoints = 0
for ff in range(numframes):
    timestamp = time.time()
    clip = hdmi_in.readframe()
    timeReadingFrames += (time.time() - timestamp)
    
    # Only do the processing every 5th frame, else just pass the frame through
    # this increases framerate from ~6 FPS to ~25
    # TODO: get the box to stay for an extra two frames after
    # TODO: Hardware accelerate
    # TODO: make the bubbles actually form around enemies
    if (ff % 5 == 0):
        timestamp = time.time()
        blurred = process_image(clip)
        timeBlurring += (time.time() - timestamp)

        timestamp = time.time()
        keypoints = detector.detect(blurred)
        timeKeypoints += (time.time() - timestamp)

        timestamp = time.time()
        clip_with_keypoints = cv2.drawKeypoints(blurred, keypoints, clip, (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        timeDrawingPoints += (time.time() - timestamp)

            #cv2.imshow("Keypoints", clip)

        outframe = hdmi_out.newframe()
        outframe[:] = clip
        hdmi_out.writeframe(outframe)
    else:
        hdmi_out.writeframe(clip)
    
end = time.time()
print("total time: " + str(end-start))
print("Frames per second:  " + str(numframes / (end - start)))
print("time reading frames: " + str(timeReadingFrames))
print("time blurring : " + str(timeBlurring))
print("time keypoints: " + str(timeKeypoints))
print("Time drawing points: " + str(timeDrawingPoints))

total time: 21.755008935928345
Frames per second:  22.06388429504532
time reading frames: 4.31354022026062
time blurring : 1.7280464172363281
time keypoints: 10.98567819595337
Time drawing points: 0.3894500732421875


In [8]:
# https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/notebooks/video/opencv_filters_hdmi.ipynb

import cv2
import numpy as np
import time
numframes = 960
grayscale = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
blurred = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width), dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, 
                           hdmi_in.mode.width), dtype=np.uint8)

start = time.time()
timestamp = time.time()
timeBW = 0
timeGauss = 0
timeCanny = 0
timeColor = 0

for ff in range(numframes):
    inframe = hdmi_in.readframe()
    
    
    if (ff % 5 == 0):
        timestamp = time.time()
        cv2.cvtColor(inframe,cv2.COLOR_RGB2GRAY,dst=grayscale)
        timeBW += (time.time() - timestamp)

        inframe.freebuffer()

        timestamp = time.time()
        cv2.GaussianBlur(grayscale,(5,5),0,dst=blurred)
        timeGauss += (time.time() - timestamp)

        timestamp = time.time()
        cv2.Canny(grayscale, 100, 110, edges=result)
        timeCanny += (time.time() - timestamp)
    
        outframe = hdmi_out.newframe()

        timestamp = time.time()
        cv2.cvtColor(result, cv2.COLOR_GRAY2RGB,dst=outframe)
        timeColor += (time.time() - timestamp)

        hdmi_out.writeframe(outframe)
    else :
        hdmi_out.writeframe(inframe)
end = time.time()
print("total time: " + str(end-start))
print("Frames per second:  " + str(numframes / (end - start)))
print("time to B&W: " + str(timeBW))
print("time Gaussian blur : " + str(timeGauss))
print("time canny: " + str(timeCanny))
print("Time back to clr: " + str(timeColor))

total time: 27.43881583213806
Frames per second:  34.9869325947947
time to B&W: 1.093994140625
time Gaussian blur : 1.7678158283233643
time canny: 9.295008659362793
Time back to clr: 0.6358826160430908


In [4]:
hdmi_out.close()
hdmi_in.close()